In [3]:
from pulp import *
# initialize the model
prob = LpProblem("aavinMilk2", LpMaximize)

supply = {
    0: 223350,
    1: 249689,
    2: 232867,
    3: 249313
    }

demand = {
    0: 275672,
    1: 308182,
    2: 287419,
    3: 307718
    }
extra_reconstitution_cost = 2
butter_cost = 112
smp_cost = 163
other_supplier_cost = 16.5
normal_cost = 15
rmo_limit = 25000 # as per Sir

T = len(demand)
# ---------------------
# VARIABLES
# ---------------------

rm = LpVariable.dicts("rm", range(0, T), 0, None)
rmo = LpVariable.dicts("rmo", range(0, T), 0, None) 

rcm = LpVariable.dicts("rcm", range(0, T), 0, None)
output = LpVariable.dicts("output", range(0, T), 0, None)
butterop = LpVariable.dicts("butterop", range(0, T), 0, None)
smpop = LpVariable.dicts("smpop", range(0, T), 0, None)

smp = LpVariable.dicts("smp", range(0, T), 0, None)
butter = LpVariable.dicts("butter", range(0, T ), 0, None) 

isbutterpr = LpVariable.dicts("isbutterpr", range(0, T), 0, 1, cat=pulp.LpBinary)
issmppr = LpVariable.dicts("issmppr", range(0, T), 0, 1, cat=pulp.LpBinary)

isbutterop = LpVariable.dicts("isbutterop", range(0, T), 0, 1, cat=pulp.LpBinary)
issmpop = LpVariable.dicts("issmpop", range(0, T), 0, 1, cat=pulp.LpBinary)
isbutter = LpVariable.dicts("isbutter", range(0, T), 0, 1, cat=pulp.LpBinary)
issmp = LpVariable.dicts("issmp", range(0, T), 0, 1, cat=pulp.LpBinary)

cfwd = LpVariable.dicts("cfwd", range(0, T ), 0, None) 
cfsmp = LpVariable.dicts("cfsmp", range(0, T), 0, None)
cfbutter = LpVariable.dicts("cfbutter", range(0, T), 0, None)

usedbutter = LpVariable.dicts("usedbutter", range(0, T), 0, None)
usedsmp = LpVariable.dicts("usedsmp", range(0, T), 0, None)


# Constraints
prob += cfwd[0] ==  0.05 * 269299 + 0.05 * 25000 #12110 #
prob += cfsmp[0] == 1200
prob += cfbutter[0] == 520

prob += rm[0] ==  0# prior period raw milk
prob += rmo[0] ==  0# prior period raw milk
prob += smp[0] == 0
prob += butter[0] == 0
prob += output[0] == 0
prob += rcm[0] == 0
prob += butterop[0] == 0
prob += smpop[0] == 0

prob += isbutterpr[0] == 0
prob += issmppr[0] == 0
prob += isbutterop[0] == 0
prob += issmpop[0] == 0
prob += isbutter[0] == 0
prob += issmp[0] == 0

prob += usedbutter[0] == 0
prob += usedsmp[0] == 0

for i in range(1, T):
    prob += cfwd[i-1] + .95*rm[i] + .95*rmo[i] + 10.7 * usedsmp[i] + 18.24 * usedbutter[i] == \
    output[i] # change this = 10.7, 18.24
    prob += output[i] <= demand[i-1]
    prob += cfwd[i] == .05*rm[i] + .05*rmo[i]
    prob += rm[i] <= supply[i-1]
    
    prob +=  54.87 * usedsmp[i] ==  93.5 * usedbutter[i] # change this
    prob +=  cfbutter[i] <= 10000
    prob +=  cfbutter[i] <= 1000 * isbutter[i]
    
    
    prob += cfbutter[i] == cfbutter[i-1] + butter[i] - usedbutter[i]
    prob += cfsmp[i] == cfsmp[i-1] + smp[i] - usedsmp[i]
    prob += rmo[i] <= rmo_limit
  

# Objective function
prob += sum([22 * output[i] - 163 * smp[i] - 112 * butter[i] - 2500 * isbutter[i] - \
             19 * rm[i] - 20.5 * rmo[i] - \
             21 * 10 * (usedsmp[i] + 2 * usedbutter[i]) for i in range(1,T)]), "Objective"
    
    
prob.writeLP("aavinMilk2.lp")
    
status = prob.solve(GLPK(options=["--ranges","aavinMilk2.sen"]))
print(status)

#print the result
print("rm")
for i in range(1, T):
    print(rm[i].value())


print("rmo")       
for i in range(1, T):
    print(rmo[i].value())

print("isbutter")   
for i in range(1, T):
    print(isbutter[i].value())

    
print("butter")   
for i in range(1, T):
    print(butter[i].value())
    
print("smp")    
for i in range(1, T):
    print(smp[i].value())

print("cfw")       
for i in range(1, T):
    print(cfwd[i].value())

print("cfbutter")   
for i in range(1, T):
    print(cfbutter[i].value())
    
print("cfsmp")    
for i in range(1, T):
    print(cfsmp[i].value())

    
print("usedsmp")    
for i in range(1, T):
    print(usedsmp[i].value())

print("usedbutter")    
for i in range(1, T):
    print(usedbutter[i].value())

print("output")    
for i in range(1, T):
    print(output[i].value())
    
print("Objective", value(prob.objective))

1
rm
223350.0
249689.0
232867.0
rmo
25000.0
25000.0
25000.0
isbutter
0
0
0
butter
0.0
0.0
0.0
smp
0.0
0.0
0.0
cfw
12417.5
13734.5
12893.4
cfbutter
0.0
0.0
0.0
cfsmp
313.906
313.906
313.906
usedsmp
886.094
0.0
0.0
usedbutter
520.0
0.0
0.0
output
267009.0
273372.0
258708.0
Objective 2225764.26


In [25]:
#%load aavinMilk.sen
